In [ ]:
import pandas as pd
from enceladus.workflows import TestingPipeline

worker = TestingPipeline(
    data_dir='/home/cam/Documents/database_tools/data/mimic3-data-20230405/data/',
    model_dir='heartfelt/enceladus-fine-tuning/model-vulcan-dominion-2:v0',
    scaler_dir='/home/cam/Documents/database_tools/data/mimic3-data-20230405/records_info_1680799732178117604.pkl',
)

# returns rescaled data
error, ppg, vpg, apg, abp, pred = worker.run()

In [ ]:
import pickle as pkl

with open('/home/cam/Documents/database_tools/data/mimic3-data-20230405/records_info_1680799732178117604.pkl', 'rb') as f:
    scalers, _ = pkl.load(f)
scalers

# breezy-wind-80:v8

### MIMIC-III 30k Test Results (max, min method)

|   Error  | Systolic | Diastolic |
|   -:     | -:       | -:        |
| < 15mmHg | 72.6%    | 89.5%     |
| < 10mmHg | 59.1%    | 80.2%     |
| <  5mmHg | 36.1%    | 59.4%     |

### VitalDB 30k Test Results

|   Error  | Systolic | Diastolic |
|   -:     | -:       | -:        |
| < 15mmHg | %    | %     |
| < 10mmHg | %    | %     |
| <  5mmHg | %    | %     |

# breezy-wind-80:v8

### MIMIC-III 30k Test Results (peak, valley method)

|   Error  | Systolic | Diastolic |
|   -:     | -:       | -:        |
| < 15mmHg | 76.9%    | 77.9%     |
| < 10mmHg | 63.5%    | 70.4%     |
| <  5mmHg | 40.5%    | 53.9%     |

In [ ]:
import pandas as pd
import plotly.graph_objects as go

pd.options.plotting.backend = 'plotly'

fig = go.Figure()
fig.add_trace(go.Histogram(x=error['sbp_err'], name='SBP Error'))
fig.add_trace(go.Histogram(x=error['dbp_err'], name='DBP Error'))

fig.update_layout(
        barmode='overlay',
        title={
            'text': 'Model Prediction Error Histogram',
            'font': {'size': 20},
        },
        xaxis={'title': 'Error (mmHg)', 'titlefont':{'size': 15}},
        yaxis={'title': 'Number of Samples', 'titlefont':{'size': 15}},
        font={
            'family': 'Courier New, monospace',
            'color' : '#FFFFFF',
            'size'  : 12,
        },
        template='plotly_dark',
        width=800,
)
fig
fig.update_traces(opacity=0.75)

In [ ]:
import plotly.graph_objects as go
from random import randint
from scipy.signal import find_peaks

fig = go.Figure()
i = randint(0, 29999)

peaks = find_peaks(abp[i], distance=50)[0]

fig.add_scatter(x=peaks, y=abp[i][peaks], mode='markers')
fig.add_scatter(y=abp[i], name='true')
fig.add_scatter(y=pred[i], name='pred')

In [ ]:
from plotly.subplots import make_subplots

i = 18
ppg, vpg, apg, abp, pred = None

fig = make_subplots(rows=2, cols=1, start_cell='top-left')

# Add inputs
fig.add_scatter(
    y=ppg[i, :],
    name='PPG',
    row=1,
    col=1,
)
fig.add_scatter(
    y=vpg[i, :] * 10,
    name='VPG',
    row=1,
    col=1,
)
fig.add_scatter(
    y=apg[i, :] * 50,
    name='APG',
    row=1,
    col=1,
)

# Add ABP and prediction
fig.add_scatter(
    y=abp[i, :],
    name='ABP (Ground Truth)',
    row=2,
    col=1
)
fig.add_scatter(
    y=pred[i, :],
    name='Prediction',
    row=2,
    col=1,
)

fig.update_layout(
        title={
            'text': 'Model Inputs (PPG, VPG, APG) and Module Output w/ True ABP',
            'font': {'size': 20},
        },
        xaxis2={'title': 'Time (samples)', 'titlefont':{'size': 15}},
        yaxis={'title': 'Amplitude (V)', 'titlefont':{'size': 15}},
        yaxis2={'title': 'Amplitude (mmHg)', 'titlefont':{'size': 15}},
        font={
            'family': 'Courier New, monospace',
            'color' : '#FFFFFF',
            'size'  : 12,
        },
        template='plotly_dark',
        width=800,
)
fig